# Registration across Macaque MRI templates

**Authors:** Nikoloz Sirmpilatze (nsirmpilatze@dpz.eu) & Chris Klink (c.klink@nin.knaw.nl)                

**Last updated:** Feb 2020     

**Requirements:**    
* _python_ >= 3.7
* _nipype_ >= 1.2.0
* _nilearn_ >= 0.5.2
    * Used only for visualisation
* _nibabel_ >= 2.3.3
* _joblib_ >= 0.14.1
    * Used only for parallel processing, not necessary when registrations are done serially
* _ANTs_ >= 2.1.0
    * _antsRegistration_,  _antsApplyTransforms_ and _antsAverageImages_ need to be in your path as executables    

**Citation**: Sirmpilatze, Nikoloz and Klink, P. Christiaan (2020). RheMAP: Non-linear warps between common rhesus macaque brain templates (Version 1)[Data set]. Zenodo. https://doi.org/10.5281/zenodo.3668510    

The following templates are used: 
1. [NMT v1.2](https://github.com/jms290/NMT)
2. [NMT v1.3](https://github.com/jms290/NMT)
3. [D99](https://afni.nimh.nih.gov/Macaque)
4. [INIA19](https://www.nitrc.org/projects/inia19/https://www.nitrc.org/projects/inia19/)
5. [MNI macaque](http://www.bic.mni.mcgill.ca/ServicesAtlases/Macaque)
6. [Yerkes19](https://github.com/Washington-University/NHPPipelines)

Within this notebook, they are abbreviated as *NMTv12*, *NMTv13*, *D99*, *INIA*, *MNI*, and *YRK*     

**NB!** We do not provide copies of the actual templates (licenses often forbids redistribution), but instead suggest you follow the links above and get them at the source. We do offer the warp files and warped templates that will be produced by this workflow. They can be downloaded from Zenodo (DOI:10.5281/zenodo.3668510). 

If you want to emulate this code, you can consider setting up your templates in the following folder structure:    

|--- RheMAP   
&emsp; |--- notebooks     
&emsp; |--- templates     
&emsp; &emsp; |--- D99     
&emsp; &emsp; &emsp; |--- D99_atlas_1.2a.nii.gz     
&emsp; &emsp; &emsp; |--- D99_atlas_1.2a_in_MNI.nii.gz     
&emsp; &emsp; &emsp; |--- D99_template.nii.gz      
&emsp; &emsp; |--- INIA   
&emsp; &emsp; &emsp; |--- inia19-t1-brain_truncated.nii.gz      
&emsp; &emsp; |--- MNI     
&emsp; &emsp; &emsp; |--- macaque_25_model-MNI_brain.nii.gz      
&emsp; &emsp; |--- NMT     
&emsp; &emsp; &emsp; |--- NMT_v1.2     
&emsp; &emsp; &emsp; &emsp; |--- NMT_SS.nii.gz      
&emsp; &emsp; &emsp; |--- NMT_v1.3     
&emsp; &emsp; &emsp; &emsp; |--- NMT_SS.nii.gz     
&emsp; &emsp; |--- YRK      
&emsp; &emsp; &emsp; |--- MacaqueYerkes19_T1w_0.5mm_brain.nii.gz       

After downloading the warp files and warped templates from [Zenodo](https://doi.org/10.5281/zenodo.3668510), we suggest you include them like this:      

|--- RheMAP   
&emsp; |--- notebooks     
&emsp; |--- templates     
&emsp; |--- warps       
&emsp; &emsp; |--- final     
&emsp; &emsp; &emsp; |--- D99_to_INIA_CompositeWarp.nii.gz     
&emsp; &emsp; &emsp; |--- D99_to_MNI_CompositeWarp.nii.gz      
&emsp; &emsp; &emsp; |--- etc   
&emsp; &emsp; |--- linear     
&emsp; &emsp; &emsp; |--- D99_to_INIA_affine_0GenericAffine.mat     
&emsp; &emsp; &emsp; |--- D99_to_MNI_affine_0GenericAffine.mat     
&emsp; &emsp; &emsp; |--- etc   
&emsp; &emsp; |--- nonlinear     
&emsp; &emsp; &emsp; |--- D99_to_INIA_1InverseWarp.nii.gz     
&emsp; &emsp; &emsp; |--- D99_to_INIA_1Warp.nii.gz     
&emsp; &emsp; &emsp; |--- D99_to_MNI_1InverseWarp.nii.gz     
&emsp; &emsp; &emsp; |--- D99_to_MNI_1Warp.nii.gz  
&emsp; &emsp; &emsp; |--- etc   
&emsp; |--- warped_templates       
&emsp; &emsp; |--- final     
&emsp; &emsp; &emsp; |--- D99_in_INIA_composite.nii.gz     
&emsp; &emsp; &emsp; |--- D99_in_MNI_composite.nii.gz       
&emsp; &emsp; &emsp; |--- etc   
&emsp; &emsp; |--- linear     
&emsp; &emsp; &emsp; |--- D99_in_INIA_linear.nii.gz    
&emsp; &emsp; &emsp; |--- D99_in_MNI_linear.nii.gz          
&emsp; &emsp; &emsp; |--- etc   
&emsp; &emsp; |--- nonlinear     
&emsp; &emsp; &emsp; |--- D99_in_INIA_linear+SyN.nii.gz     
&emsp; &emsp; &emsp; |--- D99_in_MNI_linear+SyN.nii.gz     
&emsp; &emsp; &emsp; |--- etc  

Based on the warp files and warped templates you could of course reconstruct the original templates with something like the following.     

On the command line:     
```bash
antsApplyTransforms -i <TEMPLATE1_in_TEMPLATE2_composite.nii.gz> \
                    -r <TEMPLATE2_in_TEMPLATE1.nii.gz> \        
                    -o <RECONSTRUCTED_ORIGINAL_TEMPLATE1.nii.gz> \
                    -t [<TEMPLATE1_to_TEMPLATE2_CompositeWarp>,1] \
                    -n Linear \
                    -d 3
```    

In NiPype:     
```python
import nipype.interfaces.ants as ants    
ants.ApplyTransforms(
            input_image=<TEMPLATE1_in_TEMPLATE2_composite.nii.gz>,
            reference_image=<TEMPLATE2_in_TEMPLATE1.nii.gz>,        
            output_image=<RECONSTRUCTED_ORIGINAL_TEMPLATE1.nii.gz>,
            transforms=<TEMPLATE1_to_TEMPLATE2_CompositeWarp>,
            invert_transform_flags=True,
            interpolation='Linear',
            dimension=3)
```

The end goal is to generate warp between each unique pair of the above 6 templates.
In total there are 15 pairs, and for each of them 2 warps are needed:
* forwards (A to B, e.g. *NMTv1.2_to_D99*)
* backwards (B to A, e.g. *D99_to_NMTv1.2*)

![warps](RegisterTemplates.png)

## Step 0: Preparations

### 0a. Import required libraries

In [ ]:
import os
import glob
import time
import shutil as sh
import nibabel as nb

from itertools import combinations
from matplotlib import pyplot as plt

import nipype.interfaces.fsl as fsl # nipype interface for FSL
import nipype.interfaces.ants as ants # nipype interface for ANTs
from nilearn.plotting import plot_anat # Plotting function from nilearn

### 0b. Define relative paths to template files

The skull-stripped isotropic volumetric images are used for registration.
* For **NMT**, **D99**, and **YRK** the provided skull-stripped brains were used
* In **INIA** the brain stem extends further down the spinal cord compared to the other temlplates, so the braisn stem was truncated at a level similar to the others
* No skull-stripped image was provided with **MNI**, so the brain was segmented semi-manually using ITK-SNAP
* For **YRK** we used the version provided together with [NHPPipelines](https://github.com/Washington-University/NHPPipelines)

In [7]:
# ============================================
# NB! THIS COULD BE DIFFERENT FOR EVERY USER
# ============================================
BASE_path     = os.path.dirname(os.getcwd()) + '/'   # repo base folder

# these follow the directory structure as outline above
TEMPLATE_path = BASE_path + 'templates/'                   # templates base folder
NMTv12_path   = TEMPLATE_path + 'NMT/NMT_v1.2/'
NMTv13_path   = TEMPLATE_path + 'NMT/NMT_v1.3/'
D99_path      = TEMPLATE_path + 'D99/'
INIA_path     = TEMPLATE_path + 'INIA/'
MNI_path      = TEMPLATE_path + 'MNI/'
YRK_path      = TEMPLATE_path + 'YRK/'

In [ ]:
NMTv12_brain = NMTv12_path + 'NMT_SS.nii.gz'
NMTv13_brain = NMTv13_path + 'NMT_SS.nii.gz'
D99_brain    = D99_path + 'D99_template.nii.gz'
INIA_brain   = INIA_path + 'inia19-t1-brain_truncated.nii.gz'
MNI_brain    = MNI_path + 'macaque_25_model-MNI_brain.nii.gz'
YRK_brain    = YRK_path + 'MacaqueYerkes19_T1w_0.5mm_brain.nii.gz'

temp_names   = ['NMTv1.2', 'NMTv1.3', 'D99', 'INIA', 'MNI', 'YRK']
temp_brains  = [NMTv12_brain, NMTv13_brain, D99_brain, INIA_brain, MNI_brain, YRK_brain]

For some reason, the header of NMT_v1.3 has both qform_code and sform_code set to 5. This is handled poorly by nibabel (resets it to 0). Alignments will be ok, but visualisations will be off. Better change it to something like 2 ('aligned').

In [ ]:
if NMTv13_brain in  locals():
    sh.copyfile(NMTv13_brain,NMTv13_brain + '.bak'); # back up the original file
    img = nb.load(NMTv13_brain)
    hdr = img.header
    hdr.set_qform(hdr.get_qform(),code=2)
    hdr.set_sform(hdr.get_sform(),code=2)
    nb.save(img, NMTv13_brain)

### 0c. Plot template brains

In [ ]:
contrasts = [-1, 0, 0, -1, -0.5, 0] # strength of contrast in the plot_anat function ("dim")
for name, brain, con in zip(temp_names, temp_brains, contrasts):
    plot_anat(brain, display_mode='ortho', title=name,
              draw_cross=False, annotate=True, dim=con);
    plt.savefig(name + '_template.png'); # save png
    plt.draw()

### 0d. Parallel or serial    
Registration can be a time consuming process. If you have the resources, you may opt to parallelize some of the steps. You can configure that here.

In [ ]:
do_parallel = True # Set to True for parallel processing
par_numjobs = 5     # How many parallel processes? This is only used when do_parallel is True
if do_parallel is True:
    from joblib import Parallel, delayed

## Step 1: Linear registration
* This is performed using [ANTs](http://stnava.github.io/ANTs/) **Affine** registration (12 degrees of freedom: 3 translations + 3 rotations + 3 scalings + 3 shears)
* The _antsRegistration_ function is called through its [nipype wrapper](https://nipype.readthedocs.io/en/latest/interfaces/generated/interfaces.ants/registration.html#registration).
* [Anatomy of an antsRegistration call](https://github.com/ANTsX/ANTs/wiki/Anatomy-of-an-antsRegistration-call) does a good job of explaining what the various parameters mean.

### 1a. Define function

In [ ]:
def register_linear(moving, target,
                    moving_prefix, target_prefix):
    """Linearly registers moving image to target image

    Parameters
    ----------
    moving: 3D NIFTI image file
    target: 3D NIFTI image file
    moving_prefix: string
        one of 'NMTv1.2', 'NMTv1.3' 'D99', 'INIA', 'MNI', 'YRK'
        is used in the naming of output files
    target_prefix: string
        one of 'NMTv1.2', 'NMTv1.3', 'D99', 'INIA', 'MNI', 'YRK'
        is used in the naming of output files

    Return
    ------
    Linear transformation matrix
        file named as '{moving_prefix}_to_{target_prefix}_0GenericAffine.mat'
    Transformed image
        moving image transformed into target image space
        file named as: {moving_prefix}_in_{target_prefix}_linear.nii.gz
    """

    rgstr = ants.registration.Registration(
           dimension=3,
           float=False,
           fixed_image=target,
           moving_image=moving,
           output_transform_prefix='{0}_to_{1}_affine_'.format(moving_prefix, target_prefix),
           output_warped_image='{0}_in_{1}_linear.nii.gz'.format(moving_prefix, target_prefix),
           initial_moving_transform_com=0,
           winsorize_lower_quantile=0.05,
           winsorize_upper_quantile=0.95,
           interpolation='Linear',
           use_histogram_matching=[True],
           transforms=['Affine'],
           transform_parameters=[(0.1,)],
           metric=['MI'],
           metric_weight=[1],
           radius_or_number_of_bins=[32],
           sampling_strategy=['Regular'],
           sampling_percentage=[0.2],
           number_of_iterations=[[1000, 500, 250, 100]],
           convergence_threshold=[1e-6],
           convergence_window_size=[10],
           shrink_factors=[[8, 4, 2, 1]],
           smoothing_sigmas=[[3, 2, 1, 0]],
           sigma_units=['vox'])
    
    rgstr.run()

### 1b. Apply function to register all unique template pairs

In [ ]:
if do_parallel: # parallel
    print('\nPerforming linear registration in parallel...')
    start_time = time.time()
    Parallel(n_jobs=par_numjobs, backend='multiprocessing')(
        delayed(register_linear)
        (brains[0],brains[1],names[0],names[1]) 
        for names, brains in zip(combinations(temp_names, 2), combinations(temp_brains, 2)))
    end_time = time.time()
    print('Linear registration completed in  {0:.2f} minutes'.format((end_time - start_time)/60))
        
else: # serial
    for names, brains in zip(combinations(temp_names, 2), combinations(temp_brains, 2)):
        
        # explicit here for clarity
        mov_pref = names[0]
        targ_pref = names[1]
        mov_brain = brains[0]
        targ_brain = brains[1]
    
        print('\nLinearly registering {0} to {1} template...'.format(mov_pref, targ_pref))
        start_time = time.time()
        register_linear(mov_brain, targ_brain, mov_pref, targ_pref)
        end_time = time.time()
        print('Linear registration completed in  {0:.2f} minutes'.format((end_time - start_time)/60))

### 1c. Visualize linear registration results

In [ ]:
for names, brains in zip(combinations(temp_names, 2), combinations(temp_brains, 2)):

    mov_pref = names[0]
    targ_pref = names[1]
    mov_brain = brains[0]
    targ_brain = brains[1]
    targ_index = temp_names.index(targ_pref)
    
    display = plot_anat(targ_brain, display_mode='ortho',
                        title='{0} edges on {1} volume'.format(mov_pref, targ_pref),
                        draw_cross=False, annotate=False, dim=contrasts[targ_index])
    brain = '{0}_in_{1}_linear.nii.gz'.format(mov_pref, targ_pref)
    display.add_edges(brain)
    plt.savefig('Linear_{0}_on_{1}.png'.format(mov_pref, targ_pref))
    plt.draw()

## Step 2: Non-linear registration
* Symmetric Diffeomorphic ([SyN](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2276735/)) registration is performed using [ANTs](http://stnava.github.io/ANTs/).
* The starting point for this step is the transformed image from the previous (linear) step
* The _antsRegistration_ function is called through its [nipype wrapper](https://nipype.readthedocs.io/en/latest/interfaces/generated/interfaces.ants/registration.html#registration).
* [Anatomy of an antsRegistration call](https://github.com/ANTsX/ANTs/wiki/Anatomy-of-an-antsRegistration-call) does a good job of explaining what the various parameters mean.

### 2a. Define function

In [ ]:
def register_SyN(moving, target,
                 moving_prefix, target_prefix):
    """Non-linearly registers moving image to target image using ANTs SyN

    Parameters
    ----------
    Same as register_linear function

    Return
    ------
    Non-linear forward warp
        file named as '{moving_prefix}_to_{target_prefix}_1Warp.nii.gz'
    Non-linear backward warp (inverse of the above)
        file named as '{moving_prefix}_to_{target_prefix}_1InverseWarp.nii.gz'
    Transformed image
        moving image transformed into target image space
        file named as: {moving_prefix}_in_{target_prefix}_linear+SyN.nii.gz
    """
    rgstr = ants.registration.Registration(
           dimension=3,
           float=False,
           fixed_image=target,
           moving_image=moving,
           output_transform_prefix='{0}_to_{1}_'.format(moving_prefix, target_prefix),
           output_warped_image='{0}_in_{1}_linear+SyN.nii.gz'.format(moving_prefix, target_prefix),
           initial_moving_transform_com=0,
           winsorize_lower_quantile=0.05,
           winsorize_upper_quantile=0.95,
           interpolation='Linear',
           use_histogram_matching=[True],
           transforms=['SyN'],
           transform_parameters=[(0.1,)],
           metric=['CC'],
           metric_weight=[1],
           radius_or_number_of_bins=[4],
           sampling_strategy=['Regular'],
           sampling_percentage=[0.2],
           number_of_iterations=[[500, 200, 50]],
           convergence_threshold=[1e-6],
           convergence_window_size=[10],
           shrink_factors=[[8, 4, 2]],
           smoothing_sigmas=[[3, 2, 1]],
           sigma_units=['vox'])
    
    rgstr.run()

### 2b. Apply function to register all unique template pairs    

In [ ]:
if do_parallel: # parallel
    print('\nPerforming non-linear registration in parallel...')
    start_time = time.time()
    Parallel(n_jobs=par_numjobs, backend='multiprocessing')(
        delayed(register_SyN)
        ('{0}_in_{1}_linear.nii.gz'.format(names[0], names[1]), brains[1], names[0], names[1]) 
        for names, brains in zip(combinations(temp_names, 2), combinations(temp_brains, 2)))
    end_time = time.time()
    print('Non-linear registration completed in  {0:.2f} minutes'.format((end_time - start_time)/60))
    
else: # serial
    for names, brains in zip(combinations(temp_names, 2), combinations(temp_brains, 2)):

        # explicit here for clarity
        mov_pref = names[0]
        targ_pref = names[1]
        mov_brain = '{0}_in_{1}_linear.nii.gz'.format(mov_pref, targ_pref)
        targ_brain = brains[1]
    
        print('\nNon-linearly registering {0} to {1} template...'.format(mov_pref, targ_pref))
        start_time = time.time()
        register_SyN(mov_brain, targ_brain, mov_pref, targ_pref)
        end_time = time.time()
        print('Non-linear registration completed in  {0:.2f} minutes'.format((end_time - start_time)/60))

### 2c: Visualize non-linear registration results

In [ ]:
for names, brains in zip(combinations(temp_names, 2), combinations(temp_brains, 2)):

    mov_pref = names[0]
    targ_pref = names[1]
    targ_brain = brains[1]
    targ_index = temp_names.index(targ_pref)
    
    display = plot_anat(targ_brain, display_mode='ortho',
                        title='{0} edges on {1} volume'.format(mov_pref, targ_pref),
                        draw_cross=False, annotate=False, dim=contrasts[targ_index])
    brain = '{0}_in_{1}_linear+SyN.nii.gz'.format(mov_pref, targ_pref)
    display.add_edges(brain)
    plt.savefig('Nonlinear_{0}_on_{1}.png'.format(mov_pref, targ_pref))
    plt.draw()

## Step 3: Derive composite warps by combining linear and non-linear steps
* This is done with the _antsApplyTransforms_ function, using its [nipype wrapper](https://nipype.readthedocs.io/en/latest/interfaces/generated/interfaces.ants/resampling.html#applytransforms).
* **Caution:** the order with which transforms are passed is akin to linear algebra notation, so first transform is applied last
* Both forward (templateA-to-templateB) and backward (templateB-to-templateA) warps are computed

### 3a. Define function

In [ ]:
def combine_transforms(moving, target,
                       moving_prefix, target_prefix):

    """Combines linear transform with non-linear warp to generate a composite warp

    Parameters
    ----------
    Same as register_linear function

    Return
    ------
    Composite forward warp
        file named as '{moving_prefix}_to_{target_prefix}_CompositeWarp.nii.gz'
    Composite backward warp (inverse of the above)
        file named as '{target_prefix}_to_{moving_prefix}_CompositeWarp.nii.gz'
    """
    
    forward = ants.ApplyTransforms(
            input_image=moving,
            output_image='{0}_to_{1}_CompositeWarp.nii.gz'.format(moving_prefix, target_prefix),
            reference_image=target,
            transforms=['{0}_to_{1}_1Warp.nii.gz'.format(moving_prefix, target_prefix),
                        '{0}_to_{1}_affine_0GenericAffine.mat'.format(moving_prefix, target_prefix)],
            dimension=3,
            print_out_composite_warp_file=True)
    forward.run()

    backward = ants.ApplyTransforms(
            input_image=target,
            output_image='{1}_to_{0}_CompositeWarp.nii.gz'.format(moving_prefix, target_prefix),
            reference_image=moving,
            transforms=['{0}_to_{1}_affine_0GenericAffine.mat'.format(moving_prefix, target_prefix),
                        '{0}_to_{1}_1InverseWarp.nii.gz'.format(moving_prefix, target_prefix)],
            invert_transform_flags=[True, False], # invert the linear transform
            dimension=3,
            print_out_composite_warp_file=True)
    
    backward.run()

### 3b. Apply function to generate all composite warp pairs

In [ ]:
if do_parallel: # parallel
    print('\nCreating composite warps in parallel...')
    start_time = time.time()
    Parallel(n_jobs=par_numjobs, backend='multiprocessing')(
        delayed(combine_transforms)
        (brains[0], brains[1], names[0], names[1]) 
        for names, brains in zip(combinations(temp_names, 2), combinations(temp_brains, 2)))
    end_time = time.time()
    print('Composite warps computed in {0:.2f} minutes'.format((end_time - start_time)/60))
    
else: # serial
    for names, brains in zip(combinations(temp_names, 2), combinations(temp_brains, 2)):

        # make explicit here
        mov_pref = names[0]
        targ_pref = names[1]
        mov_brain = brains[0]
        targ_brain = brains[1]

        print('\nCreating composite warps between {0} and {1} templates...'.format(mov_pref, targ_pref))
        start_time = time.time()
        combine_transforms(mov_brain, targ_brain, mov_pref, targ_pref)
        end_time = time.time()
        print('Composite warps computed in {0:.2f} minutes'.format((end_time - start_time)/60))

### 3c. Visually assess the derived composite warp pairs
* First use the derived composite warps to transform brains in both directions

In [ ]:
for names, brains in zip(combinations(temp_names, 2), combinations(temp_brains, 2)):

    mov_pref = names[0]
    targ_pref = names[1]
    mov_brain = brains[0]
    targ_brain = brains[1]
    
    forward = ants.ApplyTransforms(
            input_image=mov_brain,
            output_image='{0}_in_{1}_composite.nii.gz'.format(mov_pref, targ_pref),
            reference_image=targ_brain,
            transforms=['{0}_to_{1}_CompositeWarp.nii.gz'.format(mov_pref, targ_pref)],
            dimension=3)
    backward = ants.ApplyTransforms(
            input_image=targ_brain,
            output_image='{1}_in_{0}_composite.nii.gz'.format(mov_pref, targ_pref),
            reference_image=mov_brain,
            transforms=['{1}_to_{0}_CompositeWarp.nii.gz'.format(mov_pref, targ_pref)],
            dimension=3)

    print('\nTransforming {0} into {1} space...'.format(mov_pref, targ_pref))
    start_time = time.time()
    forward.run()
    end_time = time.time()
    print('Transformation complete in {0:.2f} minutes'.format((end_time - start_time)/60))
    print('Transforming {1} into {0} space...'.format(mov_pref, targ_pref))
    start_time = time.time()
    backward.run()
    end_time = time.time()
    print('Transformation complete in {0:.2f} minutes'.format((end_time - start_time)/60))

* Then, overlay the transformed (by composite warps) brains on target brains for visualization
* two plots (forward/backward) will be generated for each template pair

In [ ]:
for names, brains in zip(combinations(temp_names, 2), combinations(temp_brains, 2)):

    mov_pref = names[0]
    targ_pref = names[1]
    mov_brain = brains[0]
    targ_brain = brains[1]
    mov_index = temp_names.index(mov_pref)
    targ_index = temp_names.index(targ_pref)
    
    display = plot_anat(targ_brain, display_mode='ortho',
                        title='{0} edges on {1} volume'.format(mov_pref, targ_pref),
                        draw_cross=False, annotate=False, dim=contrasts[targ_index])
    brain = '{0}_in_{1}_composite.nii.gz'.format(mov_pref, targ_pref)
    display.add_edges(brain)
    plt.savefig('Composite_{0}_on_{1}.png'.format(mov_pref, targ_pref))
    plt.draw()
    
    display = plot_anat(mov_brain, display_mode='ortho',
                        title='{1} edges on {0} volume'.format(mov_pref, targ_pref),
                        draw_cross=False, annotate=False, dim=contrasts[mov_index])
    brain = '{1}_in_{0}_composite.nii.gz'.format(mov_pref, targ_pref)
    display.add_edges(brain)
    plt.savefig('Composite_{1}_on_{0}.png'.format(mov_pref, targ_pref))
    plt.draw()
    plt.show()

### 4. We will now create the average 'meta'-template in NMTv1.3 space (just because we can)
* Get all the templates (in NMTv1.3 space) & average them into a 'MetaTemplate'
* Plot the result

In [ ]:
avg = ants.AverageImages(
    dimension=3,
    output_average_image="MetaTemplate_in_NMTv1.3.nii.gz",
    normalize=True,
    images=[NMTv13_brain,
            "NMTv1.2_in_NMTv1.3_composite.nii.gz",
            "D99_in_NMTv1.3_composite.nii.gz", 
            "INIA_in_NMTv1.3_composite.nii.gz", 
            "MNI_in_NMTv1.3_composite.nii.gz",
            "YRK_in_NMTv1.3_composite.nii.gz"]);
avg.run();

plot_anat("MetaTemplate_in_NMTv1.3.nii.gz", display_mode='ortho', 
          title="MetaTemplate in NMTv1.3", draw_cross=False, annotate=True, dim=con);
plt.savefig('MetaTemplate_in_NMTv1.3.png')
plt.draw()

### 5. Clean-up
* Create subfolders for linear and non-linear warps, and for the warped volumes
* Put all files in sensible subfolders

In [ ]:
os.makedirs(BASE_path + "warps/linear" , exist_ok=True)
os.makedirs(BASE_path + "warps/nonlinear" , exist_ok=True)
os.makedirs(BASE_path + "warps/final" , exist_ok=True)

[os.rename(os.getcwd() + '/' + f, BASE_path + "warps/linear/" + f) for f in glob.glob('*_affine_*Affine.mat')];
[os.rename(os.getcwd() + '/' + f, BASE_path + "warps/nonlinear/" + f) for f in glob.glob('*_to_*_1Warp.nii.gz')];
[os.rename(os.getcwd() + '/' + f, BASE_path + "warps/nonlinear/" + f) for f in glob.glob('*_to_*_1InverseWarp.nii.gz')];
[os.rename(os.getcwd() + '/' + f, BASE_path + "warps/final/" + f) for f in glob.glob('*_to_*_CompositeWarp.nii.gz')];
[os.remove(f) for f in glob.glob('*Affine.mat')]; # spurious linear warps (generated by nonlinear)

os.makedirs(BASE_path + "warped_templates/linear" , exist_ok=True)
os.makedirs(BASE_path + "warped_templates/nonlinear" , exist_ok=True)
os.makedirs(BASE_path + "warped_templates/final" , exist_ok=True)
os.makedirs(BASE_path + "warped_templates/metatemplate" , exist_ok=True)

[os.rename(os.getcwd() + '/' +  f, BASE_path + "warped_templates/linear/" + f) for f in glob.glob('*_in_*linear.nii.gz')];
[os.rename(os.getcwd() + '/' +  f, BASE_path + "warped_templates/nonlinear/" + f) for f in glob.glob('*_in_*linear+SyN.nii.gz')];
[os.rename(os.getcwd() + '/' +  f, BASE_path + "warped_templates/final/" + f) for f in glob.glob('*_in_*composite.nii.gz')];
[os.rename(os.getcwd() + '/' +  f, BASE_path + "warped_templates/metatemplate/" + f) for f in glob.glob('MetaTemplate*.nii.gz')];

os.makedirs(BASE_path + "images/templates" , exist_ok=True)
os.makedirs(BASE_path + "images/linear_reg" , exist_ok=True)
os.makedirs(BASE_path + "images/nonlinear_reg" , exist_ok=True)
os.makedirs(BASE_path + "images/all_warp_pairs" , exist_ok=True)
os.makedirs(BASE_path + "images/metatemplate" , exist_ok=True)

[os.rename(os.getcwd() + '/' + f, BASE_path + "images/templates/" + f) for f in glob.glob('*_template.png')];
[os.rename(os.getcwd() + '/' + f, BASE_path + "images/linear_reg/" + f) for f in glob.glob('Linear*.png')];
[os.rename(os.getcwd() + '/' + f, BASE_path + "images/nonlinear_reg/" + f) for f in glob.glob('Nonlinear*.png')];
[os.rename(os.getcwd() + '/' + f, BASE_path + "images/all_warp_pairs/" + f) for f in glob.glob('Composite*.png')];
[os.rename(os.getcwd() + '/' + f, BASE_path + "images/metatemplate/" + f) for f in glob.glob('MetaTemplate*.png')];